In [26]:
# ライブラリ(Selenium)のインポート
from selenium import webdriver
from time import sleep

# ライブラリ(Pandas)のインポート
import pandas as pd
from datetime import datetime as dt

# ドライバのロード
path = 'C:\\Users\\Owner\\chromedriver'
driver = webdriver.Chrome(path)

# 指定サイトのオープン
url = 'https://finance.yahoo.co.jp/quote/6701.T/history'
driver.get(url)

# メインのデータフレームに最初のページのデータ取得
data = pd.read_html(driver.current_url, header=0)
main = data[0]

# 画面操作ならびにデータ取得
try:
    # ボタンが押下可能な限りループ処理
    while True:
        # ボタン取得
        path = '//*[@id="pagerbtm"]/ul/li[7]/button'
        btn = driver.find_element_by_xpath(path)
        
        # 最終ページでボタンが押下不可か判定
        if btn.get_attribute('disabled') != None:
            break
        else:
            # 画面全高(100%)の25%程までスクロール
            driver.execute_script('window.scrollTo(0, document.body.scrollHeight * 0.25);')
            sleep(1)
            # 次画面遷移
            btn.click()
            sleep(1)
            # データ取得と結合
            data = pd.read_html(driver.current_url, header=0)
            main = main.append(data[0], ignore_index=True)
except Exception:
    print('例外発生')

# 欠損値(NaN)を削除
main.dropna(inplace=True)

# 日付型のフィールド「修正日付」を追加
main['修正日付'] = [dt.strptime(val, "%Y年%m月%d日") for val in main['日付']]

# 不要なフィールド「日付」を削除
main.drop('日付', axis=1, inplace=True)

# フィールド「修正日付」をインデックス指定
main.set_index('修正日付', inplace=True)

# フィールド「修正日付」をキーに昇順にソート
main.sort_index(ascending=True, inplace=True)

In [27]:
main

,始値,高値,安値,終値,出来高,調整後終値*
修正日付,,,,,,
2021-09-07,6180,6200,6060,6080,1197600,6080
2021-09-08,6090,6100,5990,6040,1332200,6040
2021-09-09,5990,6030,5950,5970,1089000,5970
2021-09-10,5990,6200,5990,6200,1697400,6200
2021-09-13,6170,6200,6120,6140,816000,6140
...,...,...,...,...,...,...
2022-08-31,5060,5140,5010,5100,2281700,5100
2022-09-01,5050,5080,4990,4995,1252000,4995
2022-09-02,4990,5000,4915,4955,1177200,4955
